In [1]:
from dotenv import load_dotenv
load_dotenv()

True

Built-in tool : DuckDuckGo Search

In [3]:
from langchain_community.tools import DuckDuckGoSearchRun
search_instance = DuckDuckGoSearchRun()
results = search_instance.invoke("what is current breaking news in India")
print(results)

7 hours ago - The city's lowest minimum temperature ... 2024 · Arshdeep Singh, Manmeet Singh and Dilraj Singh struck a hat-trick each as India mauled Oman 17-0 to register their second consecutive win in the FIH Men's Junior World Cup in Chennai on Saturday ... Today Hindi News (हिंदी न्यूज़): Get latest Hindi News and breaking news in Hindi related to India, Business, Bollywood, Cricket, Education and states. पढ़ें Aaj ki taza khabar और live Hindi News Updates के साथ देश - दुनिया की ... 6 days ago - Monday, November 24, 2025 | 06:32 ... ... The Telecom Regulatory Authority of India (Trai) said it has disconnected and blacklisted more than 21 lakh mobile numbers and around one lakh entities involved in sending spam ... USA TODAY delivers current national and local news, sports, entertainment, finance, technology, and more through award-winning journalism, photos, and videos. 2 days ago - Under this, AMCs will pay these distributors 1 per cent of the first lump-sum investment or the fir

Built-in tool : Brave Search

Built-in tool : Shell

In [7]:
from langchain_community.tools import ShellTool
shell_instance = ShellTool()
# results = shell_instance.invoke("ls")
results = shell_instance.invoke("whoami")
print(results)

Executing command:
 whoami
sharadmishra



In [21]:
#since this tool is a runnable, we can perform these operations too
print(shell_instance.name)
print(shell_instance.description)   
print(shell_instance.args)

terminal
Run shell commands on this MacOS machine.
{'commands': {'anyOf': [{'type': 'string'}, {'items': {'type': 'string'}, 'type': 'array'}], 'description': 'List of shell commands to run. Deserialized using json.loads', 'title': 'Commands'}}


Custom Tools

Methods 1 : Using basic @tool decorator

In [ ]:
from langchain_core.tools import tool


three things to add to a normal function to make it a tool:
1. Add docstring -- necessary as addding it makes it easier for LLM to understand the tool
2. Add type hint -- again necessary because it makes it easier for LLM to understand the arguments
3. Add @tool decorator -- necessary then this becomes a tool, a runnable and we can call invoke() on its name.

In [15]:
@tool
def multiply_and_sqrt(x: int, y: int) -> float:
    """
    This function multiplies and then takes the sqaure root of the product
    """
    import math
    return math.sqrt(x * y)

In [23]:
result = multiply_and_sqrt.invoke({"x" : 2, "y" :3})
print(result)
print(multiply_and_sqrt.name)
print(multiply_and_sqrt.description)
print(multiply_and_sqrt.args)
print(type(result))
print(type(multiply_and_sqrt))

2.449489742783178
multiply_and_sqrt
This function multiplies and then takes the sqaure root of the product
{'x': {'title': 'X', 'type': 'integer'}, 'y': {'title': 'Y', 'type': 'integer'}}
<class 'float'>
<class 'langchain_core.tools.structured.StructuredTool'>


In [24]:
print(multiply_and_sqrt.args_schema.model_json_schema())

{'description': 'This function multiplies and then takes the sqaure root of the product', 'properties': {'x': {'title': 'X', 'type': 'integer'}, 'y': {'title': 'Y', 'type': 'integer'}}, 'required': ['x', 'y'], 'title': 'multiply_and_sqrt', 'type': 'object'}


Method 2 : StructuredTool

In [29]:
from langchain_core.tools import StructuredTool
from pydantic import BaseModel, Field


In [30]:
class InputValidation(BaseModel):
    x:int = Field(required = True, description = "The first number", gt = 0)
    y:int = Field(required = True, description = "The second number", gt = 0)

/var/folders/1c/wz8pfl212g12r3k6s922q0y80000gq/T/ipykernel_87106/719015384.py:2: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  x:int = Field(required = True, description = "The first number", gt = 0)
/var/folders/1c/wz8pfl212g12r3k6s922q0y80000gq/T/ipykernel_87106/719015384.py:3: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  y:int = Field(required = True, description = "The second number", gt = 0)


In [31]:
def multiply_and_sqrt(x, y):
    import math
    return math.sqrt(x * y)

In [32]:
tool_instance = StructuredTool.from_function(
    func = multiply_and_sqrt,
    name = "multiplication_and_square_root",
    description = "This function multiplies and then takes the sqaure root of the product",
    args_schema = InputValidation
)

In [33]:
result = tool_instance.invoke({"x" : 2, "y" :3})
print(result)

2.449489742783178


In [36]:
print(tool_instance.name)
print(tool_instance.description)
print(tool_instance.args)
print(type(result))
print(type(tool_instance))

multiplication_and_square_root
This function multiplies and then takes the sqaure root of the product
{'x': {'description': 'The first number', 'exclusiveMinimum': 0, 'required': True, 'title': 'X', 'type': 'integer'}, 'y': {'description': 'The second number', 'exclusiveMinimum': 0, 'required': True, 'title': 'Y', 'type': 'integer'}}
<class 'float'>
<class 'langchain_core.tools.structured.StructuredTool'>


Method 3 : Base Tool

In [38]:
from langchain_core.tools import BaseTool
from typing import Type
from pydantic import BaseModel, Field

In [39]:
class InputValidation(BaseModel):
    x:int  = Field(required = True, description = "The first number", gt = 0)
    y:int = Field(required = True, description = "The second number", gt = 0)

/var/folders/1c/wz8pfl212g12r3k6s922q0y80000gq/T/ipykernel_87106/2993078778.py:2: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  x:int  = Field(required = True, description = "The first number", gt = 0)
/var/folders/1c/wz8pfl212g12r3k6s922q0y80000gq/T/ipykernel_87106/2993078778.py:3: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  y:int = Field(required = True, description = "The second number", gt = 0)


In [42]:
class MultSqrt(BaseTool):
    name:str = "multiplication_and_square_root"
    description:str = "This function multiplies and then takes the sqaure root of the product"
    args_schema: Type[BaseModel] = InputValidation

    def _run(self, x, y):
        import math
        return math.sqrt(x * y)

In [43]:

tool_instance = MultSqrt()
result = tool_instance.invoke({"x" : 2, "y" : 3})
print(result)
print(tool_instance.name)
print(tool_instance.description)
print(tool_instance.args)
print(type(result))
print(type(tool_instance))

2.449489742783178
multiplication_and_square_root
This function multiplies and then takes the sqaure root of the product
{'x': {'description': 'The first number', 'exclusiveMinimum': 0, 'required': True, 'title': 'X', 'type': 'integer'}, 'y': {'description': 'The second number', 'exclusiveMinimum': 0, 'required': True, 'title': 'Y', 'type': 'integer'}}
<class 'float'>
<class '__main__.MultSqrt'>


Toolkits

In [44]:
from langchain_core.tools import tool


In [46]:
@tool
def multiply(x:int, y:int) -> int:
    """
    this function multiplies two numbers
    """
    return x * y

@tool
def square_root(x:int) -> float:
    """
    this function takes the square root of a number
    """
    import math
    return math.sqrt(x)

In [47]:
class MathToolkit:
    def get_tools(self):
        return [multiply, square_root]

In [49]:
toolkit = MathToolkit()
tools = toolkit.get_tools()

for tool_count in tools:
    print(f'{tool_count.name} --> {tool_count.description}')

multiply --> this function multiplies two numbers
square_root --> this function takes the square root of a number
